In [127]:
#By Kevin Xu
#kevin28520@gmail.com


#DATA:
    #1. cifar10(binary version):https://www.cs.toronto.edu/~kriz/cifar.html
    #2. pratrained weights (vgg16.npy):https://mega.nz/#!YU1FWJrA!O1ywiCS2IiOlUCtCpI6HTJOMrneN-Qdv3ywQP5poecM
    
# TO Train and test:
    #0. get data ready, get paths ready !!!
    #1. run training_and_val.py and call train() in the console
    #2. call evaluate() in the console to test
    
#


#! /usr/bin/env python   
# -*- coding: utf-8 -*-  

import os
import os.path

import numpy as np
import tensorflow as tf

import input_data
import VGG
import tools
import time
#
IMG_W = 32
IMG_H = 32
N_CLASSES = 10
BATCH_SIZE = 32
learning_rate = 0.01
MAX_STEP = 100   # it took me about one hour to complete the training.
IS_PRETRAIN = True


In [128]:
import pickle
import csv
import numpy as np
import cv2
from PIL import Image as im

def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict


pic = "/home/tony/Desktop/Datasets/cifar_10_batches_py/batches.meta"
data_dir = "/home/tony/Desktop/Datasets/cifar_10_batches_py/data_batch_"

meta = unpickle(pic)  #data format  -->  dictionary



 

In [129]:
train_path = [ (data_dir + str(i)) for i in np.arange(1, 5)]
val_path = "/home/tony/Desktop/Datasets/cifar_10_batches_py/data_batch_5"
test_path =  "/home/tony/Desktop/Datasets/cifar_10_batches_py/test_batch"

In [130]:
print(test_path)

x = unpickle(test_path)

print(x.keys())

/home/tony/Desktop/Datasets/cifar_10_batches_py/test_batch
dict_keys([b'batch_label', b'labels', b'data', b'filenames'])


In [131]:
x[b'data']

array([[158, 159, 165, ..., 124, 129, 110],
       [235, 231, 232, ..., 178, 191, 199],
       [158, 158, 139, ...,   8,   3,   7],
       ...,
       [ 20,  19,  15, ...,  50,  53,  47],
       [ 25,  15,  23, ...,  80,  81,  80],
       [ 73,  98,  99, ...,  94,  58,  26]], dtype=uint8)

In [150]:
def train():
    t0 = time.time()
    pre_trained_weights ='./vgg16_pretrain/vgg16.npy'
    train_log_dir = "./my_train_logs/"
    train_log_dir = './my_train_logs/'
    val_log_dir = './my_train_logs/val/'
    
    with tf.variable_scope(tf.get_variable_scope(), reuse=True): 
        x = tf.placeholder(tf.float32, shape=[None, IMG_W, IMG_H, 3])
        y_ = tf.placeholder(tf.int16, shape=[None, N_CLASSES]) 

        logits = VGG.VGG16N(x, N_CLASSES, IS_PRETRAIN)
        loss = tools.loss(logits, y_)
        accuracy = tools.accuracy(logits, y_)

        my_global_step = tf.Variable(0, name='global_step', trainable=False) 
        train_op = tools.optimize(loss, learning_rate, my_global_step)   

        saver = tf.train.Saver(tf.global_variables())
        summary_op = tf.summary.merge_all()   



        val_image_batch = unpickle(val_path)[b'data']
        val_image_batch = np.reshape(val_image_batch,(-1,3,32,32))
        val_image_batch = np.transpose(val_image_batch,[0,2,3,1])

        val_label_batch = unpickle(val_path)[b'labels']
        val_label_batch_tmp = np.max(val_label_batch)+1
        val_label_batch = np.eye(val_label_batch_tmp)[val_label_batch]
                    

        t1 = time.time()
        print("t1-t0 time = %f",t1-t0)
        
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)




        # load the parameter file, assign the parameters, skip the specific layers
        tools.load_with_skip(pre_trained_weights, sess, ['fc6','fc7','fc8'])   

        tra_summary_writer = tf.summary.FileWriter(train_log_dir, sess.graph)
        val_summary_writer = tf.summary.FileWriter(val_log_dir, sess.graph)
        
        
        t2 = time.time()
        print("t2-t1 time = %f",t2-t1)
        
        print("start training")
    
        try:

            for epoch in range(MAX_STEP):
                for batch_path in train_path:

                    
                    t3 = time.time()
                    print("t2-t1 time = %f",t3-t2)
                    
                    print("step %d"%epoch)

                    tra_images = unpickle(batch_path)[b'data']
                    tra_images = np.reshape(tra_images,(-1,3,32,32))
                    tra_images = np.transpose(tra_images,[0,2,3,1])

                    tra_labels = unpickle(batch_path)[b'labels']
                    tra_labels_tmp = np.max(tra_labels)+1
                    tra_labels = np.eye(tra_labels_tmp)[tra_labels]
                    
                    
                    _, tra_loss, tra_acc = sess.run([train_op, loss, accuracy],
                                                    feed_dict={x:tra_images, y_:tra_labels})            
                    if epoch % 50 == 0 or (epoch + 1) == MAX_STEP:                 
                        print ('Step: %d, loss: %.4f, accuracy: %.4f%%' % (epoch, tra_loss, tra_acc))
                        # summary_str = sess.run(summary_op)
                        # tra_summary_writer.add_summary(summary_str, step)

                    if epoch % 20 == 0 or (epoch + 1) == MAX_STEP :
                        val_loss, val_acc = sess.run([loss, accuracy],
                                                     feed_dict={x:val_image_batch,y_:val_label_batch})
                        print('**  Step %d, val loss = %.2f, val accuracy = %.2f%%  **' %(epoch, val_loss, val_acc))

                        # summary_str = sess.run(summary_op)
                        # val_summary_writer.add_summary(summary_str, step)

                    if epoch % 20 == 0 or (epoch + 1) == MAX_STEP:
                        checkpoint_path = os.path.join(train_log_dir, 'model.ckpt')
                        saver.save(sess, checkpoint_path, global_step=epoch)
                    t4 = time.time()
                    print("t4-t3 time = %f",t4-t3)
        except tf.errors.OutOfRangeError:
            print('Done training -- epoch limit reached')
        finally:
            sess.close()
        
        t4 = time.time()
        print("t4-t3 time = %f",t4-t3)
                    


In [ ]:
train()

t1-t0 time = %f 2.571227550506592
t2-t1 time = %f 77.40646386146545
start training
t2-t1 time = %f 0.0017774105072021484
step 0
Step: 0, loss: 4.4414, accuracy: 11.8800%
**  Step 0, val loss = 2.32, val accuracy = 9.60%  **
t4-t3 time = %f 581.3157982826233
t2-t1 time = %f 581.3176867961884
step 0
Step: 0, loss: 2.3197, accuracy: 9.6300%
**  Step 0, val loss = 2.30, val accuracy = 10.23%  **
t4-t3 time = %f 325.7335755825043
t2-t1 time = %f 907.051361322403
step 0
Step: 0, loss: 2.3012, accuracy: 10.0800%
**  Step 0, val loss = 2.30, val accuracy = 10.46%  **
t4-t3 time = %f 288.79326343536377
t2-t1 time = %f 1195.8518834114075
step 0
Step: 0, loss: 2.3013, accuracy: 10.1400%
**  Step 0, val loss = 2.30, val accuracy = 10.60%  **
t4-t3 time = %f 282.1490046977997
t2-t1 time = %f 1478.0223174095154
step 1
t4-t3 time = %f 412.4128520488739
t2-t1 time = %f 1890.4768891334534
step 1
t4-t3 time = %f